In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GITHUB_PAT = user_secrets.get_secret("GITHUB_PAT")

In [2]:
repo_url = f"https://{GITHUB_PAT}@github.com/tmtsmrsl/uconline_poc.git"

In [5]:
!git clone -b dev {repo_url}

Cloning into 'uconline_poc'...
remote: Enumerating objects: 349, done.
remote: Counting objects: 100% (349/349), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 349 (delta 197), reused 298 (delta 151), pack-reused 0 (from 0)
Receiving objects: 100% (349/349), 194.22 KiB | 1.30 MiB/s, done.
Resolving deltas: 100% (197/197), done.


In [6]:
%cd /kaggle/working/uconline_poc

/kaggle/working/uconline_poc


In [ ]:
!cp -r /kaggle/input/uconline-poc-artifact/artifact /kaggle/working/uconline_poc/artifact

In [ ]:
pip install -q -r requirements.txt

In [24]:
import os
import joblib
import json 
import torch
from dotenv import load_dotenv

from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from pymilvus.model.sparse.bm25 import BM25EmbeddingFunction
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    connections,
)

from ETL.ContentProcessor import ContentDocProcessor
from ETL.TranscriptProcessor import TranscriptDocProcessor

In [25]:
# This it the chunking option for the text processing.
CHUNK_TOKEN_SIZE = 500
CHUNK_TOKEN_OVERLAP = 50
TEXT_SPLITTER_OPTIONS = {"chunk_token_size": CHUNK_TOKEN_SIZE, "chunk_token_overlap": CHUNK_TOKEN_OVERLAP}

# We want the output as Langchain Document
RETURN_DICT = False

In [26]:
HTML_CONTENT_DIR = "artifact/emgt605/html_content"

# The CSS elements to exclude when extracting text from the HTML content
EXCLUDED_ELEMENTS_CSS='div.quiz-card__feedback, div.block-knowledge__retake-container, img, a, iframe'

# Traverse the JSON_DIR and process all the JSON files 
html_content_docs = []
json_files = [f for f in os.listdir(HTML_CONTENT_DIR) if f.endswith('.json')]
content_doc_processor = ContentDocProcessor(text_splitter_options=TEXT_SPLITTER_OPTIONS, excluded_elements_css=EXCLUDED_ELEMENTS_CSS, return_dict=RETURN_DICT)

for json_file in json_files:
    json_path = os.path.join(HTML_CONTENT_DIR, json_file)
    docs = content_doc_processor.run(json_path)
    html_content_docs.extend(docs)
    
for doc in html_content_docs:
    doc.metadata['type'] = 'html_content'

In [27]:
TRANSCRIPT_DIR = "artifact/emgt605/transcripts"

# Traverse the TRANSCRIPT_DIR and process all the transcript files
transcript_docs = []
module_dirs = os.listdir(TRANSCRIPT_DIR)
transcript_doc_processor = TranscriptDocProcessor(text_splitter_options=TEXT_SPLITTER_OPTIONS, return_dict=RETURN_DICT)

for module_dir in module_dirs:
    module_path = os.path.join(TRANSCRIPT_DIR, module_dir)
    docs = transcript_doc_processor.process_module_transcripts(module_path)
    transcript_docs.extend(docs)
    
for doc in transcript_docs:
    doc.metadata['type'] = 'video_transcript'

FileNotFoundError: [Errno 2] No such file or directory: 'artifact/emgt605/transcripts\\module_3_systems_approach_to_sustainability\\echo360/emgt605_w1_5_marginas1full_v2_mov.vtt'

In [ ]:
combined_docs = html_content_docs + transcript_docs